In [1]:
import cobra
import pandas as pd

In [11]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml")

In [3]:
model

Name,ropacus_annotated_curated
Memory address,0x0133256550
Number of metabolites,1583
Number of reactions,2383
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [4]:
glucose_fluxes = pd.read_csv('../13C_flux_data/13C_glucose_flux_data.csv')
glycolysis_fluxes = glucose_fluxes[glucose_fluxes['Pathway'] == 'Glycolysis']
glycolysis_fluxes

,Pathway,Forward Reactions,Reverse Reactions,Reversed?,Reaction,Flux,90% Confidence Lower Bound,90% Confidence Upper Bound,\
1,Glycolysis,PPGKr_reverse_3b2cc or HEX1,PPGKr or HEX1_reverse_25efa,False,G6P <-> F6P,-1.61,-2.09,1.42,NaN
2,Glycolysis,FBP or PFK_reverse_d24a6,FBP_reverse_bf2c9 or PFK,NaN,F6P + ATP -> FBP,0.00,0.00,1.91,NaN
3,Glycolysis,FBA,FBA_reverse_84806,False,FBP <-> DHAP + GAP,0.00,0.00,1.91,NaN
4,Glycolysis,TPI,TPI_reverse_c2c3b,False,DHAP <-> GAP,0.00,0.00,1.91,NaN
5,Glycolysis,GAPD and PGK_reverse_02696,GAPD_reverse_459c1 and PGK,NaN,GAP <-> 3PG + ATP + NADH,86.56,82.61,88.84,NaN
6,Glycolysis,( PGM_1 or PGM_reverse_fc9af ) and ENO,( PGM or PGM_1_reverse_02963 ) and ENO_reverse...,NaN,3PG <-> PEP,78.52,71.65,81.26,NaN
7,Glycolysis,FRUpts2pp or SBTpts or MNLpts or ACGApts or FR...,FRUpts2pp_reverse_55dac or SBTpts_reverse_74ed...,NaN,PEP <-> Pyr + ATP,19.02,-10.89,75.15,NaN


In [7]:
model.metabolites.get_by_id('glc__D_e').reactions

# print(met_1)

# for r in model.metabolites.get_by_id('glc__D_e').reactions:
#     print(r)

frozenset({<Reaction BG_CELLB at 0x133ccb810>,
           <Reaction BG_MADG at 0x133cd2410>,
           <Reaction BG_MBDG at 0x133cd9ad0>,
           <Reaction EX_glc__D_e at 0x134c43890>,
           <Reaction GLCabc at 0x134212e10>,
           <Reaction GLCpts at 0x134220d50>,
           <Reaction GLCtex at 0x134232b10>})

In [12]:
with model:
    model.objective = 'Growth_Glucose'
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    model.medium = medium
    solution = model.optimize()
    
print(solution)

<Solution 0.793 at 0x139ab6290>


In [13]:
for r in model.metabolites.get_by_id('glc__D_e').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.0 BG_MADG: h2o_e + madg_e --> glc__D_e + meoh_e
10.0 GLCpts: glc__D_e + pep_c --> g6p_c + pyr_c
0.0 BG_MBDG: h2o_e + mbdg_e --> glc__D_e + meoh_e
0.0 GLCabc: atp_c + glc__D_e + h2o_c --> adp_c + glc__D_c + h_c + pi_c
0.0 GLCtex: glc__D_e <=> glc__D_p
0.0 BG_CELLB: cellb_e + h2o_e --> 2.0 glc__D_e
-10.0 EX_glc__D_e: glc__D_e <=> 


In [14]:
for r in model.metabolites.get_by_id('g6p_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.0 AB6PGH: arbt6p_c + h2o_c --> g6p_c + hqn_c
9.781806902332953 PGI: g6p_c <=> f6p_c
0.0 PPGKr: glc__D_c + pi_c <-- g6p_c + h2o_c
10.0 GLCpts: glc__D_e + pep_c --> g6p_c + pyr_c
0.0 FFSD: h2o_c + suc6p_c --> fru_c + g6p_c
0.0 MI1PS: g6p_c --> mi1p__D_c
0.0 TRE6PS: g6p_c + udpg_c --> h_c + tre6p_c + udp_c
0.0 G6PDH2r: g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c
-0.2181930976670478 PGMT: g1p_c <=> g6p_c
0.0 S6PG: h2o_c + salcn6p_c --> 2hxmp_c + g6p_c
0.0 HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
0.0 MAL6PG: h2o_c + malt6p_c --> g6p_c + glc__D_c
0.0 GLCptspp: glc__D_p + pep_c --> g6p_c + pyr_c


In [15]:
for r in model.metabolites.get_by_id('f6p_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.368716341469047 GF6PTA: f6p_c + gln__L_c --> gam6p_c + glu__L_c
0.0 MAN6PI: man6p_c <=> f6p_c
9.781806902332953 PGI: g6p_c <=> f6p_c
0.0 M1PD: mnl1p_c + nad_c <=> f6p_c + h_c + nadh_c
0.0 FBP: fdp_c + h2o_c --> f6p_c + pi_c
0.0 FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c
0.0 G6PDA: gam6p_c + h2o_c --> f6p_c + nh4_c
0.0 PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
0.0 SBTPD: nad_c + sbt6p_c <=> f6p_c + h_c + nadh_c
-0.16450046680266062 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
8.903724302980127 F6PA: f6p_c <=> dha_c + g3p_c
0.0 FRUpts2: fru_e + pep_c --> f6p_c + pyr_c
0.0 HEX7: atp_c + fru_c --> adp_c + f6p_c + h_c


In [16]:
for r in model.metabolites.get_by_id('dha_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.0 DHAPT: dha_c + pep_c --> dhap_c + pyr_c
8.903724302980127 DHAK: atp_c + dha_c --> adp_c + dhap_c + h_c
8.903724302980127 F6PA: f6p_c <=> dha_c + g3p_c


In [17]:
for r in model.metabolites.get_by_id('dhap_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

-0.6759094244970617 G3PD1: glyc3p_c + nad_c <=> dhap_c + h_c + nadh_c
8.226060093709998 TPI: dhap_c <=> g3p_c
0.0 FBA3: s17bp_c <=> dhap_c + e4p_c
0.0 RMPA: rml1p_c <=> dhap_c + lald__L_c
0.0 DHAPT: dha_c + pep_c --> dhap_c + pyr_c
0.0 FBA2: f1p_c <=> dhap_c + glyald_c
0.0 FBA: fdp_c <=> dhap_c + g3p_c
0.0017547847730654406 QULNS: dhap_c + iasp_c --> 2.0 h2o_c + pi_c + quln_c
0.0 TGBPA: tagdp__D_c <=> dhap_c + g3p_c
0.0 MGSA: dhap_c --> mthgxl_c + pi_c
8.903724302980127 DHAK: atp_c + dha_c --> adp_c + dhap_c + h_c
0.0 INSCR: 2ins_c + atp_c --> adp_c + dhap_c + 2.0 h_c + msa_c


In [18]:
for r in model.metabolites.get_by_id('g3p_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.0 DRPA: 2dr5p_c --> acald_c + g3p_c
0.0 TRPS3: 3ig3p_c --> g3p_c + indole_c
0.0007869559199277294 DXPS: g3p_c + h_c + pyr_c --> co2_c + dxyl5p_c
16.81082623189666 GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
-0.16450046680266062 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c
8.226060093709998 TPI: dhap_c <=> g3p_c
0.00017135330514555394 PLPS: g3p_c + gln__L_c + r5p_c --> glu__L_c + 3.0 h2o_c + h_c + pi_c + pydx5p_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c
0.0 FBA: fdp_c <=> dhap_c + g3p_c
0.0 EDA: 2ddg6p_c --> g3p_c + pyr_c
-0.16450046680266062 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
0.0 TGBPA: tagdp__D_c <=> dhap_c + g3p_c
0.02686593551272348 TRPS1: 3ig3p_c + ser__L_c --> g3p_c + h2o_c + trp__L_c
8.903724302980127 F6PA: f6p_c <=> dha_c + g3p_c
0.0 DDPGALA: 2dh3dgal6p_c <=> g3p_c + pyr_c


In [20]:
for r in model.metabolites.get_by_id('13dpg_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

16.81082623189666 GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
-16.81082623189666 PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c


In [21]:
for r in model.metabolites.get_by_id('3pg_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.9040106394806502 PGCD: 3pg_c + nad_c --> 3php_c + h_c + nadh_c
-15.90681559241601 PGM: 2pg_c <=> 3pg_c
-16.81082623189666 PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
0.0 PGM_1: 3pg_c <=> 2pg_c
0.0 GLYCK: atp_c + glyc__R_c --> 3pg_c + adp_c + h_c


In [27]:
for r in model.metabolites.get_by_id('2pg_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

15.91 ENO: 2pg_c <=> h2o_c + pep_c
0.00 PGM_1: 3pg_c <=> 2pg_c
-15.91 PGM: 2pg_c <=> 3pg_c
0.00 GLYCK2: atp_c + glyc__R_c --> 2pg_c + adp_c + h_c


In [23]:
for r in model.metabolites.get_by_id('pep_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.0 TREpts: pep_c + tre_e --> pyr_c + tre6p_c
0.0 FRUpts2: fru_e + pep_c --> f6p_c + pyr_c
0.0 TREptspp: pep_c + tre_p --> pyr_c + tre6p_c
0.0 MALTptspp: malt_p + pep_c --> malt6p_c + pyr_c
3.445691484186008 PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
0.0 FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c
0.0 MNLpts: mnl_e + pep_c --> mnl1p_c + pyr_c
0.0 FRUptspp: fru_p + pep_c --> f1p_c + pyr_c
0.0 KDOPS: ara5p_c + h2o_c + pep_c --> kdo8p_c + pi_c
0.0 MALTpts: malt_e + pep_c --> malt6p_c + pyr_c
0.0 MANpts: man_e + pep_c --> man6p_c + pyr_c
0.18036532427845853 DDPA: e4p_c + h2o_c + pep_c --> 2dda7p_c + pi_c
0.0 GLCptspp: glc__D_p + pep_c --> g6p_c + pyr_c
0.0 SUCptspp: pep_c + sucr_p --> pyr_c + suc6p_c
1.9831958198062956 PPC: co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.0 PPS: atp_c + h2o_c + pyr_c --> amp_c + 2.0 h_c + pep_c + pi_c
0.0 SUCpts: pep_c + sucr_e --> pyr_c + suc6p_c
0.11719763986678855 UAGCVT: pep_c + uacgam_c --> pi_c + uaccg_c
0.0 SBTpts: pep_c + sbt__D_e --> pyr_c + sbt6p_c


In [24]:
for r in model.metabolites.get_by_id('pyr_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

0.0 ALAD_L: ala__L_c + h2o_c + nad_c --> h_c + nadh_c + nh4_c + pyr_c
0.0 SERD_D: ser__D_c --> nh4_c + pyr_c
0.0 FRUpts2: fru_e + pep_c --> f6p_c + pyr_c
0.0 L_LACD2: lac__L_c + q8_c --> pyr_c + q8h2_c
0.0 FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c
0.0 MANpts: man_e + pep_c --> man6p_c + pyr_c
0.0 PYRt4pp: na1_p + pyr_p --> na1_c + pyr_c
7.695032684777191e-05 SHCHCS: ichor_c + sucsal_c --> 2shchc_c + pyr_c
0.0 GLCptspp: glc__D_p + pep_c --> g6p_c + pyr_c
0.0 SUCptspp: pep_c + sucr_p --> pyr_c + suc6p_c
-0.8869395575110752 ALATA_L: akg_c + ala__L_c <=> glu__L_c + pyr_c
0.0 PFOR: coa_c + fldox_c + h_c + pyr_c --> accoa_c + co2_c + fldrd_c
0.0 SUCpts: pep_c + sucr_e --> pyr_c + suc6p_c
0.0 CYSTL: cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c
0.0 SBTpts: pep_c + sbt__D_e --> pyr_c + sbt6p_c
0.0 SHCHCS2: ichor_c + ssaltpp_c --> 2shchc_c + pyr_c + thmpp_c
0.0 ACGApts: acgam_e + pep_c --> acgam6p_c + pyr_c
0.0 4H2KPILY: 4h2kpi_c --> pyr_c + sucsal_c
0.0 DHAPT: dha_c + pep_c --> dhap_c + pyr

In [26]:
for r in model.metabolites.get_by_id('accoa_c').reactions:
    print(f'{solution.fluxes[r.id]:.2f}', r)

3.96 ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c
0.00 DHPACCOAHIT: 2dhphaccoa_c + coa_c + 2.0 h2o_c --> 3hadpcoa_c + accoa_c + h2_c + h_c
0.00 KAT6: 3otdcoa_c + coa_c --> accoa_c + ddcacoa_c
0.37 G1PACT: accoa_c + gam1p_c --> acgam1p_c + coa_c + h_c
-0.00 ACACT8r: 3ohodcoa_c + coa_c <=> accoa_c + pmtcoa_c
0.00 ACS: ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
0.00 FAO2: atp_c + 8.0 coa_c + 7.0 fad_c + 7.0 h2o_c + hdca_c + 7.0 nad_c --> 8.0 accoa_c + amp_c + 7.0 fadh2_c + 7.0 h_c + 7.0 nadh_c + ppi_c
0.00 ACGS: accoa_c + glu__L_c --> acglu_c + coa_c + h_c
0.00 MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c
0.00 MSHS: accoa_c + cigam_c --> coa_c + msh_c
0.00 KAT7: 3ohdcoa_c + coa_c --> accoa_c + tdcoa_c
0.00 KAS8: accoa_c + 20.0 h_c + 7.0 malcoa_c + 14.0 nadph_c --> 7.0 co2_c + 8.0 coa_c + 6.0 h2o_c + hdca_c + 14.0 nadp_c
0.00 FAO3: atp_c + 9.0 coa_c + 8.0 fad_c + 8.0 h2o_c + 8.0 nad_c + ocdca_c --> 9.0 accoa_c + amp_c + 8.0 fadh2_c + 8.0 h_c + 8.0 nadh_c + ppi_c
0.00 MACCOAT:

### TCA Cycle

In [28]:
for r in model.metabolites.get_by_id('cit_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 CITL: cit_c --> ac_c + oaa_c
0.0 CITt2r: cit_e + h_e --> cit_c + h_c
0.0 FE3DCITabcpp: atp_c + fe3dcit_p + h2o_c --> adp_c + 2.0 cit_c + fe3_c + h_c + pi_c
0.0 CITt7: cit_e + succ_c --> cit_c + succ_e
0.0 CITACt: ac_c + cit_e --> ac_e + cit_c
0.0 FEDCabc: atp_c + 2.0 cit_e + fe3_e + h2o_c --> adp_c + 2.0 cit_c + fe3_c + h_c + pi_c
0.6107483029774257 ACONTa: cit_c <=> acon_C_c + h2o_c
0.0 ACONT: cit_c <=> icit_c
0.0 CITt14: ca2_e + cit_e + h_e --> ca2_c + cit_c + h_c
0.0 CITt_kt: cit_p + h_p --> cit_c + h_c
0.6107483029774257 CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
0.0 CITt7pp: cit_p + succ_c --> cit_c + succ_p


In [29]:
for r in model.metabolites.get_by_id('acon_C_c').reactions:
    print(solution.fluxes[r.id], r)

0.6107483029774257 ACONTb: acon_C_c + h2o_c <=> icit_c
0.6107483029774257 ACONTa: cit_c <=> acon_C_c + h2o_c


In [30]:
for r in model.metabolites.get_by_id('icit_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 ICDHx: icit_c + nad_c <=> akg_c + co2_c + nadh_c
0.0 ICL: icit_c --> glx_c + succ_c
0.6107483029774257 ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
0.0 ACONT: cit_c <=> icit_c
0.6107483029774257 ACONTb: acon_C_c + h2o_c <=> icit_c
0.0 ICITt_kt: h_p + icit_p <=> h_c + icit_c


In [31]:
for r in model.metabolites.get_by_id('akg_c').reactions:
    print(solution.fluxes[r.id], r)

0.6107483029774257 ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
0.0 VALTA: akg_c + val__L_c <=> 3mob_c + glu__L_c
-1.8025924639935995 ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c
0.0 LEUTAi: 4mop_c + glu__L_c --> akg_c + leu__L_c
0.0 UACGAMOAT: glu__L_c + uacgamo_c <=> akg_c + uacgnn_c
0.0 UDPKAAT: glu__L_c + udpLa4o_c <=> akg_c + udpLa4n_c
-0.06534556384226307 TYRTA: akg_c + tyr__L_c <=> 34hpp_c + glu__L_c
-0.13980159281681606 ACOTA: acorn_c + akg_c <=> acg5sa_c + glu__L_c
0.0 AKGDa: akg_c + h_c + lpam_c <=> co2_c + sdhlam_c
-0.08756281468118755 PHETA1: akg_c + phe__L_c <=> glu__L_c + phpyr_c
0.0 OHPBAT: glu__L_c + ohpb_c <=> akg_c + phthr_c
0.9040106394806502 PSERT: 3php_c + glu__L_c --> akg_c + pser__L_c
0.0 ABTA: 4abut_c + akg_c --> glu__L_c + sucsal_c
0.0 AKGMAL: akg_c + mal__L_e <=> akg_e + mal__L_c
-0.8869395575110752 ALATA_L: akg_c + ala__L_c <=> glu__L_c + pyr_c
0.0 ORNTA_1: akg_c + orn__L_c --> glu5sa_c + glu__L_c
0.0 AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + s

In [32]:
for r in model.metabolites.get_by_id('succoa_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
0.0 3OADPCOAT: 3oxoadp_c + succoa_c --> oxadpcoa_c + succ_c
0.0 3OXCOAT: coa_c + oxadpcoa_c --> accoa_c + succoa_c
0.0 OOR3r: akg_c + coa_c + fdxox_c --> co2_c + fdxrd_c + h_c + succoa_c
0.231879244333642 HSST: hom__L_c + succoa_c --> coa_c + suchms_c
0.2793873500024739 THDPS: h2o_c + succoa_c + thdp_c --> coa_c + sl2a6o_c
7.695032684777191e-05 SSCOARy: h_c + nadph_c + succoa_c <=> coa_c + nadp_c + sucsal_c
0.0 OCOAT3r: ghb_c + succoa_c <=> 4hbutcoa_c + succ_c
0.0 AKGDb: coa_c + sdhlam_c --> dhlam_c + succoa_c
0.0 MMM2: succoa_c --> mmcoa__R_c
0.0 SUCBZT1: bz_c + 4.0 h_c + succoa_c <=> benzcoa_c + succ_c
0.0 SUCBZT2: 4hbz_c + succoa_c <=> 4hbzcoa_c + succ_c
0.0 BSCT: coa_c + 4.0 h_c + sbzcoa_c <=> benzcoa_c + succoa_c
0.0 BZSCT: bzsucc_c + succoa_c <=> bzsuccoa_c + succ_c
0.0 OCOAT2r: 3mb_c + succoa_c <=> ivcoa_c + succ_c
0.0 OCOAT1: acac_c + succoa_c --> aacoa_c + succ_c
0.5113435446629636 SUCOAS: atp_c + coa_c + succ_c <=

In [62]:
met_1 = model.metabolites.get_by_id('succ_c')
met_2 = model.metabolites.get_by_id('fum_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 SUCDi: q8_c + succ_c --> fum_c + q8h2_c
0.2546936823310454 FRD2: fum_c + mql8_c --> mqn8_c + succ_c
0.0006854132205822015 FRD3: 2dmmql8_c + fum_c --> 2dmmq8_c + succ_c
0.0 FRD5: fum_c + mql6_c --> mqn6_c + succ_c
0.0 SUCD1: fad_c + succ_c <-- fadh2_c + fum_c
0.0 FRD: fadh2_c + fum_c --> fad_c + succ_c
0.0017547847730654406 ASPO5: asp__L_c + fum_c --> h_c + iasp_c + succ_c


In [63]:
met_1 = model.metabolites.get_by_id('fum_c')
met_2 = model.metabolites.get_by_id('mal__L_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.4294595339441475 FUM: fum_c + h2o_c <=> mal__L_c


In [35]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 AKGMAL: akg_c + mal__L_e <=> akg_e + mal__L_c
0.0 MALt2_2pp: 2.0 h_p + mal__L_p --> 2.0 h_c + mal__L_c
0.0 MALtpp: h_p + mal__L_p <=> h_c + mal__L_c
0.0 ME1: mal__L_c + nad_c --> co2_c + nadh_c + pyr_c
0.4301449471647297 MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
0.0 MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
0.4294595339441475 FUM: fum_c + h2o_c <=> mal__L_c
0.0 MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
0.0006854132205822015 MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c
0.0 ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
0.0 MALt2_2: 2.0 h_e + mal__L_e --> 2.0 h_c + mal__L_c
0.0 MALt2r: h_e + mal__L_e <=> h_c + mal__L_c
0.0 DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c


In [64]:
met_1 = model.metabolites.get_by_id('mal__L_c')
met_2 = model.metabolites.get_by_id('oaa_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.4301449471647297 MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
0.0 MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
0.0 MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
0.0 DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c


In [36]:
for r in model.metabolites.get_by_id('oaa_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 DTARTD: tartr__D_c --> h2o_c + oaa_c
0.0 OAAt2_2pp: 2.0 h_p + oaa_p --> 2.0 h_c + oaa_c
1.9831958198062956 PPC: co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.0 CITL: cit_c --> ac_c + oaa_c
0.4301449471647297 MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
0.0 MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
0.0 PC: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c
0.0 MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
0.0 OAADC: h_c + oaa_c --> co2_c + pyr_c
0.0 PPCK: atp_c + oaa_c --> adp_c + co2_c + pep_c
0.0 ASPO1: asp__L_c + h2o_c + o2_c --> h2o2_c + nh4_c + oaa_c
0.6107483029774257 CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
0.0 PEPCK_re: gtp_c + oaa_c --> co2_c + gdp_c + pep_c
-1.8025924639935995 ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c
0.0 DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c


### Pentose Phosphate

In [37]:
for r in model.metabolites.get_by_id('g6p_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 AB6PGH: arbt6p_c + h2o_c --> g6p_c + hqn_c
9.781806902332953 PGI: g6p_c <=> f6p_c
0.0 PPGKr: glc__D_c + pi_c <-- g6p_c + h2o_c
10.0 GLCpts: glc__D_e + pep_c --> g6p_c + pyr_c
0.0 FFSD: h2o_c + suc6p_c --> fru_c + g6p_c
0.0 MI1PS: g6p_c --> mi1p__D_c
0.0 TRE6PS: g6p_c + udpg_c --> h_c + tre6p_c + udp_c
0.0 G6PDH2r: g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c
-0.2181930976670478 PGMT: g1p_c <=> g6p_c
0.0 S6PG: h2o_c + salcn6p_c --> 2hxmp_c + g6p_c
0.0 HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
0.0 MAL6PG: h2o_c + malt6p_c --> g6p_c + glc__D_c
0.0 GLCptspp: glc__D_p + pep_c --> g6p_c + pyr_c


In [65]:
met_1 = model.metabolites.get_by_id('g6p_c')
met_2 = model.metabolites.get_by_id('6pgl_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 G6PDH2r: g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c


In [38]:
# 6 phosphogluconolactone
for r in model.metabolites.get_by_id('6pgl_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 PGL: 6pgl_c + h2o_c --> 6pgc_c + h_c
0.0 G6PDH2r: g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c
0.0 G6PBDH: g6p_B_c + nadp_c --> 6pgl_c + h_c + nadph_c


In [66]:
met_1 = model.metabolites.get_by_id('6pgl_c')
met_2 = model.metabolites.get_by_id('6pgc_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 PGL: 6pgl_c + h2o_c --> 6pgc_c + h_c


In [41]:
# 6 phosphogluconate
for r in model.metabolites.get_by_id('6pgc_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 PGLCNDH: 6p2dhglcn_c + nadph_c <=> 6pgc_c + 2.0 h_c + nadp_c
0.0 GNK: atp_c + glcn_c --> 6pgc_c + adp_c + h_c
0.0 EDD: 6pgc_c --> 2ddg6p_c + h2o_c
0.0 PGL: 6pgl_c + h2o_c --> 6pgc_c + h_c
0.0 GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c


In [67]:
met_1 = model.metabolites.get_by_id('6pgc_c')
met_2 = model.metabolites.get_by_id('ru5p__D_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c


In [42]:
# ribulose 5 phosphate
for r in model.metabolites.get_by_id('ru5p__D_c').reactions:
    print(solution.fluxes[r.id], r)

0.0006854132205822173 DB4PS: ru5p__D_c --> db4p_c + for_c + h_c
-0.5093662578837798 RPE: ru5p__D_c <=> xu5p__D_c
-0.5086808446631975 RPI: r5p_c <=> ru5p__D_c
0.0 A5PISO: ru5p__D_c <=> ara5p_c
0.0 GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c


In [68]:
met_1 = model.metabolites.get_by_id('ru5p__D_c')
met_2 = model.metabolites.get_by_id('xu5p__D_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

-0.5093662578837798 RPE: ru5p__D_c <=> xu5p__D_c


In [70]:
met_1 = model.metabolites.get_by_id('ru5p__D_c')
met_2 = model.metabolites.get_by_id('r5p_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

-0.5086808446631975 RPI: r5p_c <=> ru5p__D_c


In [43]:
# ribose 5 phosphate
for r in model.metabolites.get_by_id('r5p_c').reactions:
    print(solution.fluxes[r.id], r)

0.6730099581607125 PRPPS: atp_c + r5p_c <=> amp_c + h_c + prpp_c
0.0 NMNN: h2o_c + nmn_c --> h_c + ncam_c + r5p_c
0.0 RBK: atp_c + rib__D_c --> adp_c + h_c + r5p_c
0.00017135330514555394 PLPS: g3p_c + gln__L_c + r5p_c --> glu__L_c + 3.0 h2o_c + h_c + pi_c + pydx5p_c
-0.5086808446631975 RPI: r5p_c <=> ru5p__D_c
0.0 ADPRDP: adprib_c + h2o_c --> amp_c + 2.0 h_c + r5p_c
-0.16450046680266062 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c


In [44]:
# xyulose 5 phosphate
for r in model.metabolites.get_by_id('xu5p__D_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 RBP4E: ru5p__L_c <=> xu5p__D_c
-0.5093662578837798 RPE: ru5p__D_c <=> xu5p__D_c
0.0 XYLK: atp_c + xylu__D_c --> adp_c + h_c + xu5p__D_c
-0.16450046680266062 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c


In [71]:
met_1 = model.metabolites.get_by_id('xu5p__D_c')
met_2 = model.metabolites.get_by_id('g3p_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

-0.16450046680266062 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c


In [45]:
for r in model.metabolites.get_by_id('g3p_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 DRPA: 2dr5p_c --> acald_c + g3p_c
0.0 TRPS3: 3ig3p_c --> g3p_c + indole_c
0.0007869559199277294 DXPS: g3p_c + h_c + pyr_c --> co2_c + dxyl5p_c
16.81082623189666 GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
-0.16450046680266062 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c
8.226060093709998 TPI: dhap_c <=> g3p_c
0.00017135330514555394 PLPS: g3p_c + gln__L_c + r5p_c --> glu__L_c + 3.0 h2o_c + h_c + pi_c + pydx5p_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c
0.0 FBA: fdp_c <=> dhap_c + g3p_c
0.0 EDA: 2ddg6p_c --> g3p_c + pyr_c
-0.16450046680266062 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
0.0 TGBPA: tagdp__D_c <=> dhap_c + g3p_c
0.02686593551272348 TRPS1: 3ig3p_c + ser__L_c --> g3p_c + h2o_c + trp__L_c
8.903724302980127 F6PA: f6p_c <=> dha_c + g3p_c
0.0 DDPGALA: 2dh3dgal6p_c <=> g3p_c + pyr_c


In [46]:
for r in model.metabolites.get_by_id('s7p_c').reactions:
    print(solution.fluxes[r.id], r)

-0.16450046680266062 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c
-0.16450046680266062 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
0.0 S7PI: s7p_c --> gmhep7p_c


In [47]:
for r in model.metabolites.get_by_id('f6p_c').reactions:
    print(solution.fluxes[r.id], r)

0.368716341469047 GF6PTA: f6p_c + gln__L_c --> gam6p_c + glu__L_c
0.0 MAN6PI: man6p_c <=> f6p_c
9.781806902332953 PGI: g6p_c <=> f6p_c
0.0 M1PD: mnl1p_c + nad_c <=> f6p_c + h_c + nadh_c
0.0 FBP: fdp_c + h2o_c --> f6p_c + pi_c
0.0 FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c
0.0 G6PDA: gam6p_c + h2o_c --> f6p_c + nh4_c
0.0 PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
0.0 SBTPD: nad_c + sbt6p_c <=> f6p_c + h_c + nadh_c
-0.16450046680266062 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
8.903724302980127 F6PA: f6p_c <=> dha_c + g3p_c
0.0 FRUpts2: fru_e + pep_c --> f6p_c + pyr_c
0.0 HEX7: atp_c + fru_c --> adp_c + f6p_c + h_c


In [48]:
for r in model.metabolites.get_by_id('e4p_c').reactions:
    print(solution.fluxes[r.id], r)

-0.16450046680266062 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
0.18036532427845853 DDPA: e4p_c + h2o_c + pep_c --> 2dda7p_c + pi_c
0.0 E4PD: e4p_c + h2o_c + nad_c <=> 4per_c + 2.0 h_c + nadh_c
0.0 FBA3: s17bp_c <=> dhap_c + e4p_c
-0.34486579108111914 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c


### ED Pathway

In [49]:
model.reactions.get_by_id('EDA')

Reaction identifier,EDA
Name,2-dehydro-3-deoxy-phosphogluconate aldolase
Memory address,0x013b1d2550
Stoichiometry,2ddg6p_c --> g3p_c + pyr_c 2-Dehydro-3-deoxy-D-gluconate 6-phosphate --> Glyceraldehyde 3-phosphate + Pyruvate
GPR,WP_005250926_1 or WP_005251224_1 or WP_005251401_1
Lower bound,0.0
Upper bound,1000.0


In [51]:
for r in model.metabolites.get_by_id('6pgc_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 PGLCNDH: 6p2dhglcn_c + nadph_c <=> 6pgc_c + 2.0 h_c + nadp_c
0.0 GNK: atp_c + glcn_c --> 6pgc_c + adp_c + h_c
0.0 EDD: 6pgc_c --> 2ddg6p_c + h2o_c
0.0 PGL: 6pgl_c + h2o_c --> 6pgc_c + h_c
0.0 GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c


In [50]:
# KDPG
for r in model.metabolites.get_by_id('2ddg6p_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 EDD: 6pgc_c --> 2ddg6p_c + h2o_c
0.0 EDA: 2ddg6p_c --> g3p_c + pyr_c
0.0 DDGLK: 2ddglcn_c + atp_c --> 2ddg6p_c + adp_c + h_c


In [74]:
# KDPG
for r in model.metabolites.get_by_id('2ddglcn_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 MNNH: mana_c --> 2ddglcn_c + h2o_c
0.0 DDGLK: 2ddglcn_c + atp_c --> 2ddg6p_c + adp_c + h_c


In [72]:
model.metabolites.get_by_id('2ddglcn_c')

Metabolite identifier,2ddglcn_c
Name,2-Dehydro-3-deoxy-D-gluconate
Memory address,0x013a213810
Formula,C6H9O6
Compartment,C_c
In 2 reaction(s),"MNNH, DDGLK"


In [73]:
model.metabolites.get_by_id('2ddg6p_c')

Metabolite identifier,2ddg6p_c
Name,2-Dehydro-3-deoxy-D-gluconate 6-phosphate
Memory address,0x013a4f8090
Formula,C6H8O9P
Compartment,C_c
In 3 reaction(s),"EDD, EDA, DDGLK"


In [75]:
model.metabolites.get_by_id('mana_c')


Metabolite identifier,mana_c
Name,D-Mannonate
Memory address,0x013a7af850
Formula,C6H11O7
Compartment,C_c
In 2 reaction(s),"MNNH, MANAO"


### Glyoxylate Shunt

In [53]:
for r in model.metabolites.get_by_id('icit_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 ICDHx: icit_c + nad_c <=> akg_c + co2_c + nadh_c
0.0 ICL: icit_c --> glx_c + succ_c
0.6107483029774257 ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
0.0 ACONT: cit_c <=> icit_c
0.6107483029774257 ACONTb: acon_C_c + h2o_c <=> icit_c
0.0 ICITt_kt: h_p + icit_p <=> h_c + icit_c


In [76]:
met_1 = model.metabolites.get_by_id('icit_c')
met_2 = model.metabolites.get_by_id('glx_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 ICL: icit_c --> glx_c + succ_c


In [54]:
for r in model.metabolites.get_by_id('glx_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 GLXCL: 2.0 glx_c + h_c --> 2h3oppan_c + co2_c
0.0 GLYCTO1: glyclt_c + o2_c --> glx_c + h2o2_c
0.0 GLYCTO2: glyclt_c + q8_c --> glx_c + q8h2_c
0.0006854132205822015 GLYCTO4: 2dmmq8_c + glyclt_c --> 2dmmql8_c + glx_c
0.0 GLYO1: gly_c + h2o_c + o2_c --> glx_c + h2o2_c + nh4_c
0.0 GLYCLTDx: glx_c + h_c + nadh_c --> glyclt_c + nad_c
0.0 UGLYCH: h2o_c + 2.0 h_c + urdglyc_c --> co2_c + glx_c + 2.0 nh4_c
0.0 GLXO1: glx_c + h2o_c + nad_c --> 2.0 h_c + nadh_c + oxa_c
0.0 GLYCTO3: glyclt_c + mqn8_c --> glx_c + mql8_c
0.0006854132205822015 MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c
0.0 ICL: icit_c --> glx_c + succ_c
0.0 FDMO6: fmnh2_c + o2_c + sulfac_c --> fmn_c + glx_c + h2o_c + h_c + so3_c
0.0 FDMO5_1: fmnRD_c + o2_c + sula_c --> fmn_c + glx_c + h2o_c + h_c + so3_c


In [77]:
met_1 = model.metabolites.get_by_id('glx_c')
met_2 = model.metabolites.get_by_id('mal__L_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0006854132205822015 MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c


In [56]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 AKGMAL: akg_c + mal__L_e <=> akg_e + mal__L_c
0.0 MALt2_2pp: 2.0 h_p + mal__L_p --> 2.0 h_c + mal__L_c
0.0 MALtpp: h_p + mal__L_p <=> h_c + mal__L_c
0.0 ME1: mal__L_c + nad_c --> co2_c + nadh_c + pyr_c
0.4301449471647297 MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
0.0 MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
0.4294595339441475 FUM: fum_c + h2o_c <=> mal__L_c
0.0 MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
0.0006854132205822015 MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c
0.0 ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
0.0 MALt2_2: 2.0 h_e + mal__L_e --> 2.0 h_c + mal__L_c
0.0 MALt2r: h_e + mal__L_e <=> h_c + mal__L_c
0.0 DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c


### Anaplerotic Pathways

In [58]:
for r in model.metabolites.get_by_id('pyr_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 ALAD_L: ala__L_c + h2o_c + nad_c --> h_c + nadh_c + nh4_c + pyr_c
0.0 SERD_D: ser__D_c --> nh4_c + pyr_c
0.0 FRUpts2: fru_e + pep_c --> f6p_c + pyr_c
0.0 L_LACD2: lac__L_c + q8_c --> pyr_c + q8h2_c
0.0 FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c
0.0 MANpts: man_e + pep_c --> man6p_c + pyr_c
0.0 PYRt4pp: na1_p + pyr_p --> na1_c + pyr_c
7.695032684777191e-05 SHCHCS: ichor_c + sucsal_c --> 2shchc_c + pyr_c
0.0 GLCptspp: glc__D_p + pep_c --> g6p_c + pyr_c
0.0 SUCptspp: pep_c + sucr_p --> pyr_c + suc6p_c
-0.8869395575110752 ALATA_L: akg_c + ala__L_c <=> glu__L_c + pyr_c
0.0 PFOR: coa_c + fldox_c + h_c + pyr_c --> accoa_c + co2_c + fldrd_c
0.0 SUCpts: pep_c + sucr_e --> pyr_c + suc6p_c
0.0 CYSTL: cyst__L_c + h2o_c --> hcys__L_c + nh4_c + pyr_c
0.0 SBTpts: pep_c + sbt__D_e --> pyr_c + sbt6p_c
0.0 SHCHCS2: ichor_c + ssaltpp_c --> 2shchc_c + pyr_c + thmpp_c
0.0 ACGApts: acgam_e + pep_c --> acgam6p_c + pyr_c
0.0 4H2KPILY: 4h2kpi_c --> pyr_c + sucsal_c
0.0 DHAPT: dha_c + pep_c --> dhap_c + pyr

In [78]:
met_1 = model.metabolites.get_by_id('pyr_c')
met_2 = model.metabolites.get_by_id('oaa_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 OAADC: h_c + oaa_c --> co2_c + pyr_c
0.0 PC: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c


In [57]:
for r in model.metabolites.get_by_id('oaa_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 DTARTD: tartr__D_c --> h2o_c + oaa_c
0.0 OAAt2_2pp: 2.0 h_p + oaa_p --> 2.0 h_c + oaa_c
1.9831958198062956 PPC: co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.0 CITL: cit_c --> ac_c + oaa_c
0.4301449471647297 MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
0.0 MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
0.0 PC: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c
0.0 MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
0.0 OAADC: h_c + oaa_c --> co2_c + pyr_c
0.0 PPCK: atp_c + oaa_c --> adp_c + co2_c + pep_c
0.0 ASPO1: asp__L_c + h2o_c + o2_c --> h2o2_c + nh4_c + oaa_c
0.6107483029774257 CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
0.0 PEPCK_re: gtp_c + oaa_c --> co2_c + gdp_c + pep_c
-1.8025924639935995 ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c
0.0 DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c


In [79]:
met_1 = model.metabolites.get_by_id('pyr_c')
met_2 = model.metabolites.get_by_id('mal__L_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
0.0 ME1: mal__L_c + nad_c --> co2_c + nadh_c + pyr_c


In [59]:
for r in model.metabolites.get_by_id('pep_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 TREpts: pep_c + tre_e --> pyr_c + tre6p_c
0.0 FRUpts2: fru_e + pep_c --> f6p_c + pyr_c
0.0 TREptspp: pep_c + tre_p --> pyr_c + tre6p_c
0.0 MALTptspp: malt_p + pep_c --> malt6p_c + pyr_c
3.445691484186008 PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
0.0 FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c
0.0 MNLpts: mnl_e + pep_c --> mnl1p_c + pyr_c
0.0 FRUptspp: fru_p + pep_c --> f1p_c + pyr_c
0.0 KDOPS: ara5p_c + h2o_c + pep_c --> kdo8p_c + pi_c
0.0 MALTpts: malt_e + pep_c --> malt6p_c + pyr_c
0.0 MANpts: man_e + pep_c --> man6p_c + pyr_c
0.18036532427845853 DDPA: e4p_c + h2o_c + pep_c --> 2dda7p_c + pi_c
0.0 GLCptspp: glc__D_p + pep_c --> g6p_c + pyr_c
0.0 SUCptspp: pep_c + sucr_p --> pyr_c + suc6p_c
1.9831958198062956 PPC: co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.0 PPS: atp_c + h2o_c + pyr_c --> amp_c + 2.0 h_c + pep_c + pi_c
0.0 SUCpts: pep_c + sucr_e --> pyr_c + suc6p_c
0.11719763986678855 UAGCVT: pep_c + uacgam_c --> pi_c + uaccg_c
0.0 SBTpts: pep_c + sbt__D_e --> pyr_c + sbt6p_c


In [80]:
met_1 = model.metabolites.get_by_id('pep_c')
met_2 = model.metabolites.get_by_id('oaa_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

1.9831958198062956 PPC: co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.0 PEPCK_re: gtp_c + oaa_c --> co2_c + gdp_c + pep_c
0.0 PPCK: atp_c + oaa_c --> adp_c + co2_c + pep_c


### Acetate Formation

In [82]:
met_1 = model.metabolites.get_by_id('accoa_c')
met_2 = model.metabolites.get_by_id('ac_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 ACS: ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
0.0 HXCT: accoa_c + hxa_c --> ac_c + hxcoa_c
0.0 BUTCT: accoa_c + but_c --> ac_c + btcoa_c
0.0 ACACCT: acac_c + accoa_c --> aacoa_c + ac_c


### 1-Carbon metabolism

### Oxidative Phosphorylation

In [60]:
for r in model.metabolites.get_by_id('nad_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 FADRx: fad_c + h_c + nadh_c --> fadh2_c + nad_c
0.0 UACGAMDH: nad_c + uacgam_c <=> h_c + nadh_c + uacgamo_c
0.0 LCARS: h_c + lald__L_c + nadh_c <=> 12ppd__S_c + nad_c
0.0 MMSAD2: coa_c + nad_c + ppal_c <=> h_c + nadh_c + ppcoa_c
0.0 GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c
0.0 ALDD3: h2o_c + nad_c + ppal_c --> 2.0 h_c + nadh_c + ppa_c
0.0 MTHFR2_1: 2.0 h_c + mlthf_c + nadh_c --> 5mthf_c + nad_c
0.0 ALDD31_1: amacald_c + h2o_c + nad_c --> gly_c + h_c + nadh_c
0.0 EAR140x: h_c + nadh_c + tmrs2eACP_c --> myrsACP_c + nad_c
0.0 XMO: h_c + nadh_c + o2_c + tol_c --> bzalc_c + h2o_c + nad_c
0.0 HACD7i: 3hhdcoa_c + nad_c --> 3ohdcoa_c + h_c + nadh_c
0.0 MTHFR2: 2.0 h_c + mlthf_c + nadh_c --> 5mthf_c + nad_c
0.0 EAR80x: h_c + nadh_c + toct2eACP_c --> nad_c + ocACP_c
0.0 MTHFD2i: mlthf_c + nad_c --> methf_c + nadh_c
0.0 ALDD2x: acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
0.0 GLUSx: akg_c + gln__L_c + h_c + nadh_c --> 2.0 glu__L_c + nad_c
0.0 OXPTNDH: h2o_c + n

In [83]:
met_1 = model.metabolites.get_by_id('nadh_c')
met_2 = model.metabolites.get_by_id('atp_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 FAO2: atp_c + 8.0 coa_c + 7.0 fad_c + 7.0 h2o_c + hdca_c + 7.0 nad_c --> 8.0 accoa_c + amp_c + 7.0 fadh2_c + 7.0 h_c + 7.0 nadh_c + ppi_c
0.0 FAO11: atp_c + 8.0 coa_c + 6.0 fad_c + 7.0 h2o_c + hdcea_c + 7.0 nad_c --> 8.0 accoa_c + amp_c + 6.0 fadh2_c + 7.0 h_c + 7.0 nadh_c + ppi_c
0.0 FAO3: atp_c + 9.0 coa_c + 8.0 fad_c + 8.0 h2o_c + 8.0 nad_c + ocdca_c --> 9.0 accoa_c + amp_c + 8.0 fadh2_c + 8.0 h_c + 8.0 nadh_c + ppi_c
0.0 FAO10: atp_c + 9.0 coa_c + 7.0 fad_c + 8.0 h2o_c + 8.0 nad_c + ocdcea_c --> 9.0 accoa_c + amp_c + 7.0 fadh2_c + 8.0 h_c + 8.0 nadh_c + ppi_c
0.0 FAO1: atp_c + 7.0 coa_c + 6.0 fad_c + 6.0 h2o_c + 6.0 nad_c + ttdca_c --> 7.0 accoa_c + amp_c + 6.0 fadh2_c + 6.0 h_c + 6.0 nadh_c + ppi_c


In [61]:
for r in model.metabolites.get_by_id('fadh2_c').reactions:
    print(solution.fluxes[r.id], r)

-0.006825642078693854 ACOAD6f: fad_c + tdcoa_c <=> fadh2_c + td2coa_c
0.0 CHOLD2: chol_c + fad_c --> betald_c + fadh2_c
0.0 FADRx: fad_c + h_c + nadh_c --> fadh2_c + nad_c
0.0 PROD2: fad_c + pro__L_c --> 1pyr5c_c + fadh2_c + h_c
0.0 FAO2: atp_c + 8.0 coa_c + 7.0 fad_c + 7.0 h2o_c + hdca_c + 7.0 nad_c --> 8.0 accoa_c + amp_c + 7.0 fadh2_c + 7.0 h_c + 7.0 nadh_c + ppi_c
0.0 FADRx2: fad_c + h_c + nadph_c --> fadh2_c + nadp_c
0.0 BZSCD: bzsuccoa_c + fad_c <-- fadh2_c + phitcoa_c
0.0 S2FE2SS2: atp_c + fadh2_c + 2.0 fe2_c + h2o_c + sufbcd_2fe2s_c + 2.0 sufsesh_c --> adp_c + fad_c + 7.0 h_c + pi_c + sufbcd_2fe2s2_c + 2.0 sufse_c
0.0 FAO3: atp_c + 9.0 coa_c + 8.0 fad_c + 8.0 h2o_c + 8.0 nad_c + ocdca_c --> 9.0 accoa_c + amp_c + 8.0 fadh2_c + 8.0 h_c + 8.0 nadh_c + ppi_c
0.0 SUCD3: fadh2_c + q_c --> fad_c + qh2_c
0.0 ACOAD4f: dcacoa_c + fad_c <=> dc2coa_c + fadh2_c
-0.006825642078693854 ACOAD2f: fad_c + hxcoa_c <=> fadh2_c + hx2coa_c
-0.006825642078693854 ACOAD1f: btcoa_c + fad_c <=> b2coa_c + 

In [84]:
for r in model.metabolites.get_by_id('atp_c').reactions:
    print(solution.fluxes[r.id], r)

840890466177 UMPK: atp_c + ump_c <=> adp_c + udp_c
0.0 PHEMEabcpp: atp_c + h2o_c + pheme_c --> adp_c + h_c + pheme_p + pi_c
0.0 CYTK2: atp_c + dcmp_c <=> adp_c + dcdp_c
0.0 ILEabcpp: atp_c + h2o_c + ile__L_p --> adp_c + h_c + ile__L_c + pi_c
0.0 NDPK9: atp_c + idp_c --> adp_c + itp_c
0.0 OXACOAL: atp_c + coa_c + oxa_c --> amp_c + oxalcoa_c + ppi_c
0.0 PGP180abcpp: atp_c + h2o_c + pgp180_c --> adp_c + h_c + pgp180_p + pi_c
0.0 PFK_2: atp_c + tag6p__D_c --> adp_c + h_c + tagdp__D_c
0.0 2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> amp_c + pg160_c + ppi_c
0.0 PGP160abcpp: atp_c + h2o_c + pgp160_c --> adp_c + h_c + pgp160_p + pi_c
0.344671017274146 PRASCSi: 5aizc_c + asp__L_c + atp_c --> 25aics_c + adp_c + h_c + pi_c
0.0 INSTabc: atp_c + h2o_c + inost_e --> adp_c + h_c + inost_c + pi_c
0.0 Kabc: atp_c + h2o_c + k_e --> adp_c + h_c + k_c + pi_c
0.00017135330514555394 TMPK: atp_c + thmmp_c --> adp_c + thmpp_c
0.0 XYLK: atp_c + xylu__D_c --> adp_c + h_c + xu5p__D_c
0.04477623463773884 ATPPRT: a

In [85]:
met_1 = model.metabolites.get_by_id('o2_c')
met_2 = model.metabolites.get_by_id('atp_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

In [86]:
model.metabolites.get_by_id('o2_c')

Metabolite identifier,o2_c
Name,O2
Memory address,0x0139e2f290
Formula,O2
Compartment,C_c
In 73 reaction(s),"HGNTOR, 3HPPPNH, DESAT18, FAS181, ASPO1, LYSMO, XMO, NODOy, CYO1b, FDMO2_1, FDMO6, TAUDO, FDMO4_1, FDMOtau, CPPPGO, FDMO_1, PTOLDOX, FAS161, 4HOXPACMOF, 4MCAT23DOX, PXMO, PYDXO, PYDXNO, 3HCINNMH,..."


### Transhydrogenation

In [87]:
met_1 = model.metabolites.get_by_id('nadh_c')
met_2 = model.metabolites.get_by_id('nadph_c')

for r in met_1.reactions:
    if met_2 in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 FRNDPR2r: fdxr_42_c + h_c + nadh_c + 2.0 nadp_c --> fdxo_42_c + nad_c + 2.0 nadph_c
0.0 NADTRHD: nad_c + nadph_c --> nadh_c + nadp_c
0.0 FLNDPR2r: 2.0 flxr_c + h_c + nadh_c + 2.0 nadp_c <=> 2.0 flxso_c + nad_c + 2.0 nadph_c


### ATP Maintenance

In [88]:
model.reactions.get_by_id('ATPM')

Reaction identifier,ATPM
Name,R_ATPM
Memory address,0x01380ba710
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP + H2O --> ADP + H+ + Phosphate
GPR,spontaneous
Lower bound,0.0
Upper bound,1000.0


### Transport

In [89]:
for r in model.metabolites.get_by_id('o2_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 HGNTOR: hgentis_c + o2_c --> 4mlacac_c + h_c
0.0 3HPPPNH: 3hpppn_c + h_c + nadh_c + o2_c --> dhpppn_c + h2o_c + nad_c
0.0 DESAT18: h_c + nadph_c + o2_c + stcoa_c --> 2.0 h2o_c + nadp_c + odecoa_c
0.0 FAS181: 4.0 h_c + hdca_c + malcoa_c + 3.0 nadph_c + o2_c --> co2_c + coa_c + 3.0 h2o_c + 3.0 nadp_c + ocdcea_c
0.0 ASPO1: asp__L_c + h2o_c + o2_c --> h2o2_c + nh4_c + oaa_c
0.0 LYSMO: lys__L_c + o2_c --> 5apentam_c + co2_c + h2o_c + h_c
0.0 XMO: h_c + nadh_c + o2_c + tol_c --> bzalc_c + h2o_c + nad_c
0.0 NODOy: nadph_c + 2.0 no_c + 2.0 o2_c --> h_c + nadp_c + 2.0 no3_c
0.0 CYO1b: 2.0 focytc_c + 0.5 o2_c --> 2.0 ficytc_c + h2o_c
0.0 FDMO2_1: buts_c + fmnRD_c + o2_c --> btal_c + fmn_c + h2o_c + h_c + so3_c
0.0 FDMO6: fmnh2_c + o2_c + sulfac_c --> fmn_c + glx_c + h2o_c + h_c + so3_c
0.0 TAUDO: akg_c + o2_c + taur_c --> aacald_c + co2_c + so3_c + succ_c
0.0 FDMO4_1: fmnRD_c + o2_c + taur_c --> amacald_c + fmn_c + h2o_c + h_c + so3_c
0.0 FDMOtau: fmnh2_c + o2_c + taur_c --> aacald_c + fmn_c

In [90]:
model.reactions.get_by_id('O2tpp')

Reaction identifier,O2tpp
Name,O2 transport via diffusion (periplasm)
Memory address,0x013d1a7b90
Stoichiometry,o2_p <=> o2_c O2 <=> O2
GPR,spontaneous
Lower bound,-1000.0
Upper bound,1000.0


In [93]:
for r in model.metabolites.get_by_id('co2_c').reactions:
    print(solution.fluxes[r.id], r)

0.6107483029774257 ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
0.0 FASm240: 3.0 h_c + mbhn_c + mmcoa__S_c + 2.0 nadph_c --> co2_c + coa_c + dmlgnc_c + h2o_c + 2.0 nadp_c
0.0 FAS140: ddca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + ttdca_c
0.01450448941722444 KAS4: 2mbcoa_c + 14.0 h_c + 5.0 malcoa_c + 10.0 nadph_c --> 5.0 co2_c + 6.0 coa_c + fa4_c + 4.0 h2o_c + 10.0 nadp_c
0.0 GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c
0.00725224470861222 KAS12: 2mbcoa_c + 17.0 h_c + 6.0 malcoa_c + 12.0 nadph_c --> 6.0 co2_c + 7.0 coa_c + fa12_c + 5.0 h2o_c + 12.0 nadp_c
0.008532052598367318 KAS3: 14.0 h_c + ivcoa_c + 5.0 malcoa_c + 10.0 nadph_c --> 5.0 co2_c + 6.0 coa_c + fa3_c + 4.0 h2o_c + 10.0 nadp_c
0.0021330131495918294 KAS6: 17.0 h_c + ibcoa_c + 6.0 malcoa_c + 12.0 nadph_c --> 6.0 co2_c + 7.0 coa_c + fa6_c + 5.0 h2o_c + 12.0 nadp_c
0.0 LYSMO: lys__L_c + o2_c --> 5apentam_c + co2_c + h2o_c + h_c
0.00017135330514555394 THZPSN: atp_c +

In [95]:
for r in model.metabolites.get_by_id('co2_e').reactions:
    print(solution.fluxes[r.id], r)

-10.843184515007664 CO2t: co2_e <=> co2_c
10.843184515007664 EX_co2_e: co2_e --> 


In [101]:
for r in model.metabolites.get_by_id('nh4_e').reactions:
    print(solution.fluxes[r.id], r)

0.0 NH4tex: nh4_e <=> nh4_p
7.758291044104775 NH4t: nh4_e <=> nh4_c
-7.758291044104775 EX_nh4_e: nh4_e <=> 


In [99]:
for r in model.metabolites.get_by_id('so4_e').reactions:
    print(solution.fluxes[r.id], r)

0.1200489083264562 SO4t2: h_e + so4_e --> h_c + so4_c
-0.1200489083264562 EX_so4_e: so4_e <=> 
0.0 SULabc: atp_c + h2o_c + so4_e --> adp_c + h_c + pi_c + so4_c
0.0 SO4tex: so4_e <=> so4_p


### Biomass Equation

In [92]:
for r in model.reactions:
    if 'Growth' in r.id:
        print(solution.fluxes[r.id], r.id)

0.0 Growth
0.0 Growth_Phenol
0.7933023386368239 Growth_Glucose


In [100]:
model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_phenol_e': 1000.0}

In [14]:
for m in model.metabolites:
    print(m.name)
    if 'glucose' in m.name:
        print(m)

idylglycerol (ditetradecanoyl, n-C14:0)
1,2-ditetradec-7-enoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (ditetradec-7-enoyl, n-C14:1)
1,2-dihexadecanoyl-sn-glycerol 3-phosphate
1,2-dihexadec-9-enoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (dihexadec-9-enoyl, n-C16:1)
1,2-dioctadecanoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (dioctadecanoyl, n-C18:0)
1,2-dioctadec-11-enoyl-sn-glycerol 3-phosphate
Phosphatidylglycerol (dioctadec-11-enoyl, n-C18:1)
Fe3+
Cytosine
CMP
CMP
CO2
Coniferol
Coniferyl aldehyde
Ferulate
Co2+
Ferredoxin (oxidized) 2[4Fe-4S]
Ferredoxin (reduced) 2[4Fe-4S]
Undecaprenyl phosphate
Coniferol
Coniferol
Carbon monoxide
Cyclic pyranopterin monophosphate
Coproporphyrinogen III
Protoporphyrinogen IX
P-Cresol
P-Cresol
L-Carnitine
L-Carnitine
L-Carnitinyl-CoA
Gamma-butyrobetaine
Crotonobetaine
Crotonobetainyl-CoA
D-Carnitine
D-Carnitine
Gamma-butyrobetaine
Chromate
Chromate
Uracil
Cytosine
Spermidine
Crotonobetaine
Crotonobetaine
Tetradecenoyl-CoA (n-C14:1CoA)
U

In [21]:
model.reactions.get_by_id('PGI').metabolites

{<Metabolite g6p_c at 0x139060c50>: -1.0,
 <Metabolite f6p_c at 0x1390f4110>: 1.0}

In [67]:
for r in model.metabolites.get_by_id('2pg_c').reactions:
    print(r)

ENO: 2pg_c <=> h2o_c + pep_c
PGM: 2pg_c <=> 3pg_c
PGM_1: 3pg_c <=> 2pg_c
GLYCK2: atp_c + glyc__R_c --> 2pg_c + adp_c + h_c


In [80]:
met_1 = model.metabolites.get_by_id('pep_c')
met_2 = model.metabolites.get_by_id('pyr_c')

forward_string = ''
reverse_string = ''
for r in met_1.reactions:
    if met_2 in r.metabolites:
        if met_1 in r.reactants:
            forward_string += r.id + ' or '
            reverse_string += r.reverse_id + ' or '
            # print(r)
        else:
            forward_string += r.reverse_id + ' or '
            reverse_string += r.id + ' or '

print(forward_string)
    # for m in r.metabolites:
    #     print(m.name)
        # if m.name == 'g6p_c':
        # # print(m)
        #     print(r.metabolites)
        #     print()

FRUpts2pp or SBTpts or MNLpts or ACGApts or FRUpts2 or MANpts or FRUpts or ARBTpts or PYK or GALTpts or SUCpts or TREptspp or TREpts or GLCptspp or PPS_reverse_1c319 or ACGAptspp or SALCptspp or MALTptspp or FRUptspp or SUCptspp or DHAPT or MALTpts or ACMANApts or GAMpts or SALCpts or GLCpts or 


In [68]:
model.reactions.get_by_id('ENO').reverse_id

'ENO_reverse_40eea'

In [ ]:
metabolite_name = ''
reactions = [r for r in model.reactions if 